In [6]:
import os
import glob
import keras
from keras_video import VideoFrameGenerator
import keras_video.utils
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,MaxPooling2D)
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from collections import deque
from keras.regularizers import l2
from keras.layers import BatchNormalization
from keras.utils.vis_utils import plot_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import os.path
import time

Using TensorFlow backend.


# Loading the Model From the saved JSON File

In [8]:
# Set the path to the saved location of the JSON file. 

with open('F:/DLProj/v4_1/train/Model/v1.json', 'r') as json_file:
    json_savedModel= json_file.read()
#load the model architecture 
model_j = keras.models.model_from_json(json_savedModel)
model_j.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 98, 98, 32)    4736      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 15, 96, 96, 32)    9248      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 96, 96, 32)    128       
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 96, 96, 32)    0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 15, 48, 48, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 15, 48, 48, 64)    18496     
_________________________________________________________________
time_distributed_7 (TimeDist (None, 15, 48, 48, 64)   

In [9]:
# Load the saved model weight
model_j.load_weights('F:/DLProj/v4_1/train/Weight/v1_41.hdf5')

# Defining the video generator for the test set

In [23]:
import os
import glob
import keras
from keras_video import VideoFrameGenerator

# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('F:/DLProj/v5_2/test/*')]
classes.sort()
# some global params
SIZE = (300, 300)
CHANNELS = 3
NBFRAME = 15
BS = 5
# pattern to get videos and classes
glob_pattern='F:/DLProj/v5_2/test/{classname}/*.mp4'
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
test = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=0.00001,  # 33% of the videos will be used as validation set. 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

class door, validation count: 0, train count: 16
class not_door, validation count: 0, train count: 15
Total data: 2 classes for 31 files for train


In [19]:
optimizer= RMSprop(lr=1e-4)
model_j.compile(optimizer=optimizer , 
              loss='categorical_crossentropy',
              #loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
histos = model_j.evaluate_generator(test,
                                   verbose = 1)

6/6 [==============================] - 19s 3s/step


In [28]:
print('Test score:', histos[0])
print('Test accuracy:', histos[1])

Test score: 1.0280625820159912
Test accuracy: 0.6666666865348816


In [30]:
model_j.predict_classes

<bound method Sequential.predict_classes of <keras.engine.sequential.Sequential object at 0x00000194A5875608>>